In [9]:
import getpass
import os

from langchain.chat_models import init_chat_model
from langchain_postgres import PGVector

In [8]:
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

model = init_chat_model("google_genai:gemini-2.5-flash-lite")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
